In [ ]:
import numpy as np
import skimage
import pandas as pd
import matplotlib.pyplot as plt
from utils import *
np.seterr(divide='ignore', invalid='ignore')
from datasetGenerator import DatasetGenerator
from dataLoader import DataLoader
from skimage.feature import local_binary_pattern
import cv2 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from skimage import feature
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import confusion_matrix
import seaborn as sns


In [ ]:
"""Get the dataset and extract features"""
dl = DataLoader()
dg = DatasetGenerator(dl)
"""This method is relatively slow. You can download pre-computed dataset here
https://drive.google.com/file/d/1SGJWgt3z6S44uo6GI9sFuX_WGvOoZ-8z/view?usp=sharing
"""
dg.prepareDataset()
img = cv2.imread("./dataset/images/20220000.png",0)

f1, t1, t2 = dg.extract_features_targets_pairs()

In [ ]:
"""Delete features with NaN values"""
f1, t1, t2 = np.array(f1), np.array(t1), np.array(t2)
print(f1.shape, t1.shape, t2.shape)
f1 = pd.DataFrame(f1)
nan = f1.isnull().any(axis=1)
f1 = f1[~nan]
t1 = t1[~nan]
t2 = t2[~nan]
print(f1.shape, t1.shape, t2.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(f1, t1, test_size=0.33, random_state=42)

In [ ]:
# Define a Scikit-learn Classifier and train it
from sklearn.metrics import classification_report
plt.rcParams["figure.figsize"] = (7,7)
clf = KNeighborsClassifier(n_neighbors=1,metric="cosine")
#clf = RandomForestClassifier(n_estimators=1000, max_depth=10)
clf.fit(X_train, y_train)
lab =["BackGround","C","Dead","T","Weapon"]
sns.heatmap(confusion_matrix(y_test, clf.predict(X_test)), fmt="d",cbar=False,annot=True,xticklabels=lab,yticklabels=lab,cmap="viridis")
plt.title("Confusion Matrix on ROIs test | Accuracy: {:.2f}".format(clf.score(X_test, y_test)))
plt.show()
print(classification_report(y_test, clf.predict(X_test)))


for off in range(4):
    print("Predict for offset #{}".format(off+1))
    reg = GradientBoostingRegressor()
    reg.fit(f1, t2[:,off])
    print(np.array(list(zip(reg.predict(f1[:15]),t2[:15,off]))))
    print("\n=========\n")

In [ ]:
plt.rcParams["figure.figsize"] = (15,15)
def visualize(data_dict):
    labels = {1: 'C', 2: 'dead', 3: 'T', 4: 'weapon', 0: 'background'}
    colors = {1: (0, 0, 255), 2: (255, 255, 255), 3: (255, 0, 0), 4: (255, 255, 0), 0: (0, 0, 0)}
    image = data_dict['image']
    classes = data_dict['classes']
    bboxes = data_dict['bboxes']
    for i in range(len(classes)):
        x1,y1,x2,y2 = bboxes[i,:]
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color=colors[classes[i]], thickness=2)
        cv2.putText(image,labels[classes[i]] , (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color=colors[classes[i]], thickness=2)
    plt.imshow(image)
    plt.axis('off')
    

def update_dict(data_dict,finals,img_g):
    for f_box in finals:
        p1 = f_box[:2]
        p2 = f_box[2:]
        y_min, x_min, y_max, x_max = min(p1[0], p2[0]), min(p1[1], p2[1]), max(p1[0], p2[0]), max(p1[1], p2[1])
        if (y_max-y_min) >  3.2 *(x_max-x_min) or (x_max-x_min) > 3.2*(y_max-y_min) or (y_max-y_min)*(x_max-x_min) < 2000:
            continue
        feat = dg.extract_features_roi(img_g,y_min, x_min, y_max, x_max)
        cls = int(clf.predict([feat])[0])
        
        data_dict["bboxes"].append(f_box)
        data_dict["classes"].append(cls)
    return data_dict

In [ ]:
import cv2
"""
Object detection on test images
"""
img_index = 0
for image in dl.getAllTestData():
    img_index += 1
    candidates = extract_candidates(image)
    features = []
    img_g = cv2.cvtColor((image*255).astype(np.uint8),cv2.COLOR_RGB2GRAY)/255.
    
    
    features = []
    for candidate in candidates:
        p1 = candidate[:2]
        p2 = candidate[2:]
        y_min, x_min, y_max, x_max = min(p1[0], p2[0]), min(p1[1], p2[1]), max(p1[0], p2[0]), max(p1[1], p2[1])
        features.append(dg.extract_features_roi(img_g,y_min, x_min, y_max, x_max))
            
    classes = clf.predict(features)
    classes = np.array(list(map(int,classes)))
    
        
    _sw, _sh = 2,2
    I = 1

    plt.subplot(_sw, _sh, I)
    I += 1
    plt.imshow(image.copy())
    plt.axis('off')
    plt.title("Input image #{}".format(img_index))


    data_dict = {'image':image.copy(),"classes":[],"bboxes":[]}
    finals = candidates[classes != 0]
    data_dict = update_dict(data_dict,finals,img_g)
    data_dict["classes"] = np.array(data_dict["classes"])
    data_dict["bboxes"] = np.array(data_dict["bboxes"])
    plt.subplot(_sw, _sh, I)
    I += 1
    visualize(data_dict)
    plt.title("All ROIs with detected classes")


    data_dict = {'image':image.copy(),"classes":[],"bboxes":[]}
    for label in range(1,5):
        finals = non_max_suppression(candidates[classes == label])
        data_dict = update_dict(data_dict,finals,img_g)
    data_dict["classes"] = np.array(data_dict["classes"])
    data_dict["bboxes"] = np.array(data_dict["bboxes"])
    plt.subplot(_sw, _sh, I)
    I += 1
    visualize(data_dict)
    plt.title("Non max suppression on ROIs based on predicted classes")



    data_dict = {'image':image.copy(),"classes":[],"bboxes":[]}
    finals = non_max_suppression(candidates[classes != 0])
    data_dict = update_dict(data_dict,finals,img_g)
    data_dict["classes"] = np.array(data_dict["classes"])
    data_dict["bboxes"] = np.array(data_dict["bboxes"])
    plt.subplot(_sw, _sh, I)
    I += 1
    visualize(data_dict)
    plt.title("Non max suppression on all ROIs that detected as non-background")

    plt.tight_layout()
    plt.savefig("test_{}-RF.png".format(img_index))
    plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (7,7)
sns.set_style("darkgrid")
lab =["BackGround","C","Dead","T","Weapon"]
plt.hist(t1)
plt.title("Histogram of samples",loc="center")
